<a href="https://colab.research.google.com/github/AnilKumar-06/FineFoodReview/blob/main/Amazon_Food__Reviews_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Amazon Fine Food Reviews Analysis


Dataset Source:https://www.kaggle.com/snap/amazon-fine-food-reviews

This consist reviews of fine food of Amazon

Total reviews : 568,454
Number of users: 256,059
Number of products: 74,258
Timespan: Oct 1999 - Oct 2012
Number of Attributes/Columns in data: 10

Attribute Information:



1.  Id
2.  Product-id(Unique identifier for the product)

1.   UserId

1.   Profile Name

1.   HelpfulnessNumerator-no.of users those who found the review helpful

1.   HelpfulnessDenominator - number of users those who indicated whether they found the review helpful or not

1.   Score - rating(1 to 5)

1.   Time - timestamp for the review

1.  summary - brief summary of the review

2.   Text - text of the review








**Objective:**
Given a review, determine whether the review positive or negative

We use the Score/Rating. A rating 4 or 5 will be considerd a positive review and a rating of 1 or 2 negative. Areview of rating 3 is neutral and ignored. This is an approximate and proxy way of determining the polarity(positivity or negativity) of a review.

**Loading Data**
data is available in .csv and SQLite Dataset

In order to load dataset i use SQLite dataset as it is easy to query the data and visualise the data efficiency.

we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score id above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [6]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
import sqlite3
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc 
from nltk.stem.porter import PorterStemmer

In [7]:
#using the Sqlite table to read data
con = sqlite3.connect("/content/drive/My Drive/ML_Projects/database.sqlite")

In [8]:
#filtering only positive and negative reviews not considering score = 3
filtered_data = pd.read_sql_query("""SELECT *FROM  Reviews WHERE Score != 3 limit 20000""", con)
filtered_data.head()
filtered_data.shape

(20000, 10)

In [9]:
def partition(x):
  if x<3:
    return 'negative'
  else:
    return 'positive'

In [10]:
#changing reviews with score less than 3 to be negative and vice versa
actual_score = filtered_data['Score']
positive_negative = actual_score.map(partition)
filtered_data['Score'] = positive_negative

In [11]:
filtered_data.shape #checking dimensions


(20000, 10)

In [12]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


**EDA**



Data Cleaning:Deduplication

some reviews have many duplicate entries. so we have to remove duplicate reviews in order to get unbiased result.


In [13]:
#data Cleaning Dediplication(remove duplicate reviews)
display = pd.read_sql_query("""SELECT *FROM Reviews WHERE Score != 3 AND UserId = "A395BORC6FGVXV" ORDER BY ProductId""",con)
display.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text


As we can see above same user has multiple reviews with the same values of HelpfulnessNumerator, HelpfulnessDenominator, score, Time, summary and text and on doing analysis it was found that


after analysis that reviews with the same parameter other than productId belong to the same product just having different flavour or quantity. Hence in order to reduce redundency we eliminated the rows having same parameter.

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=A395BORC6FGVXV remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product

In [14]:
#sorting by product id
sorted_data = filtered_data.sort_values('ProductId', axis = 0, ascending = True)

In [15]:
#deduplication of entries
final = sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep = 'first', inplace = False)

In [16]:
#check Remaining data in %
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

96.77

In [17]:
display = pd.read_sql_query("""SELECT *FROM Reviews WHERE Score != 3 AND Id = "44737" or Id = "64422" """,con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
1,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


In [18]:
final  = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
print(final.shape)

(19354, 10)


In [19]:
#count of each rating(review)
final['Score'].value_counts()

positive    16339
negative     3015
Name: Score, dtype: int64

In [20]:
i=0;
for sent in final['Text'].values:
  if(len(re.findall('<.*?>', sent))):
    print(i)
    print(sent)
    break;
  i += 1;

1
Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [21]:
!pip install -q wordcloud
import wordcloud


In [22]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [23]:
#import re
import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

#set of stopwords and initializing the snowball stemmer
stop = set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english')

def cleanHtml(sentence):
  cleaner = re.compile('<.*?>')
  cleantext = re.sub(cleaner, ' ', sentence)
  return cleantext


def cleanPunc(sentence):
  cleaned = re.sub(r'[?|!|\'|"|#]', r'',sentence)
  cleaned = re.sub(r'[.|,|)|(|\|/]', r' ',cleaned)
  return cleaned

print(stop)
print("\n*******************************************")
print("Base word of tasty = ",sno.stem('tasty'))

{'few', 'being', 'wouldn', 'him', 'and', 'off', 'how', 'll', 'very', "aren't", 'are', 'than', 'which', 'shan', 'while', 'if', 'here', 'ourselves', 'same', "weren't", 'with', "won't", 'didn', 's', 'don', "wasn't", 'an', "doesn't", 'couldn', 'why', 'd', 'ain', 'your', 'she', 'after', 'both', 't', 'my', 'was', 'from', 'not', "hasn't", 'any', "shan't", "shouldn't", 're', 'only', 'hers', 'did', 'own', 'them', 'has', 'doing', 'herself', 'needn', 'am', 'up', 'the', 'doesn', 'at', 'some', 'out', 'through', "you're", 'before', 'her', 'we', 'had', 'further', "needn't", 'because', 'above', 'his', 'to', 'then', 'yourself', 'yourselves', 'does', "don't", 'more', 'their', 'do', 'mightn', 'so', 'me', 'a', 'haven', 'weren', 'those', 'been', 'most', "isn't", 'aren', 'no', "you've", 'as', 'into', 'its', 'by', 'other', 'wasn', 'himself', 'there', "haven't", 'what', 'our', 'hadn', 'ours', 'over', "should've", 'in', 'on', 'can', 'myself', 'have', 'too', 'you', 'were', 'won', 'once', 'such', "you'll", 'betw

In [24]:
i=0
str1 = ' '
final_string = []
all_positive_word = []
all_negative_word = []
s = ''
for sent in final['Text'].values:
  filtered_sentence = []
  sent = cleanHtml(sent)
  for w in sent.split():
    for cleaned_words in cleanPunc(w).split():
      if ((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
        if (cleaned_words.lower() not in stop):
          s = (sno.stem(cleaned_words.lower())).encode('utf8')
          filtered_sentence.append(s)
          if(final['Score'].values)[i] == 'positive':
            all_positive_word.append(s)
          if (final['Score'].values)[i] == 'negative':
            all_negative_word.append(s)
          else:
            continue
        else:
          continue
  str1 = b" ".join(filtered_sentence)
  final_string.append(str1)
  i += 1


In [25]:
final['cleanedText'] = final_string


In [26]:
final.head(3)
print(final.shape)


(19354, 11)


In [27]:
final=final.sample(n=10000,replace="False")
print(final.shape)


(10000, 11)


In [28]:
conn = sqlite3.connect('final.sqlite')
c = conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema=None, if_exists='replace')

In [29]:
freq_dist_pos = nltk.FreqDist(all_positive_word)
freq_dist_neg = nltk.FreqDist(all_negative_word)
print("Most Common Positive word : ",freq_dist_pos.most_common(20))
print("Most Common Negative word : ",freq_dist_neg.most_common(20))

Most Common Positive word :  [(b'like', 7632), (b'tast', 6394), (b'good', 6097), (b'love', 6070), (b'flavor', 5710), (b'great', 5539), (b'one', 5187), (b'coffe', 5104), (b'use', 5056), (b'tri', 4676), (b'product', 4600), (b'food', 4213), (b'dog', 3820), (b'make', 3778), (b'get', 3747), (b'tea', 3729), (b'would', 3102), (b'buy', 2924), (b'time', 2856), (b'realli', 2842)]
Most Common Negative word :  [(b'like', 1676), (b'tast', 1663), (b'product', 1463), (b'one', 1044), (b'would', 1029), (b'flavor', 1029), (b'tri', 947), (b'food', 820), (b'use', 794), (b'dog', 767), (b'good', 756), (b'coffe', 755), (b'buy', 750), (b'get', 732), (b'order', 654), (b'dont', 617), (b'even', 600), (b'eat', 572), (b'bag', 569), (b'make', 559)]


In [30]:
#bi-grma and n-gram
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range = (1,2))
final_bigram_counts = count_vect.fit_transform(final['Text'].values) 


In [31]:
final_bigram_counts.get_shape()

(10000, 215633)

In [32]:
#TF-IDF

tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [33]:
final_tf_idf.get_shape()

(10000, 215633)

In [34]:
features = tf_idf_vect.get_feature_names()
len(features)

215633

In [35]:
features[100000:100010]

['just expensive',
 'just eye',
 'just fantastic',
 'just fedex',
 'just feed',
 'just feel',
 'just feels',
 'just felt',
 'just few',
 'just figure']

In [36]:
print(final_tf_idf[3, :].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [37]:
def top_tfidf_feats(row, features, top_n = 25):
  top_ids = np.argsort(row)[::-1][:top_n]
  top_feats = [(features[i], row[i]) for i in top_ids]
  df = pd.DataFrame(top_feats)
  df.columns = ['feature', 'tfidf']
  return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0], features, 25)

In [38]:
top_tfidf

,feature,tfidf
0,love the,0.177108
1,juices,0.160851
2,the texture,0.148203
3,seeds,0.143703
4,in cleaning,0.126485
5,keeping health,0.126485
6,much benefit,0.126485
7,heard bunch,0.126485
8,felt much,0.126485
9,of strength,0.126485


In [39]:
from gensim.models import word2vec
from gensim.models import KeyedVectors
import pickle

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/ML_Projects/GoogleNews-vectors-negative300.bin', binary=True)


In [40]:
model.wv['computer']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([ 1.07421875e-01, -2.01171875e-01,  1.23046875e-01,  2.11914062e-01,
       -9.13085938e-02,  2.16796875e-01, -1.31835938e-01,  8.30078125e-02,
        2.02148438e-01,  4.78515625e-02,  3.66210938e-02, -2.45361328e-02,
        2.39257812e-02, -1.60156250e-01, -2.61230469e-02,  9.71679688e-02,
       -6.34765625e-02,  1.84570312e-01,  1.70898438e-01, -1.63085938e-01,
       -1.09375000e-01,  1.49414062e-01, -4.65393066e-04,  9.61914062e-02,
        1.68945312e-01,  2.60925293e-03,  8.93554688e-02,  6.49414062e-02,
        3.56445312e-02, -6.93359375e-02, -1.46484375e-01, -1.21093750e-01,
       -2.27539062e-01,  2.45361328e-02, -1.24511719e-01, -3.18359375e-01,
       -2.20703125e-01,  1.30859375e-01,  3.66210938e-02, -3.63769531e-02,
       -1.13281250e-01,  1.95312500e-01,  9.76562500e-02,  1.26953125e-01,
        6.59179688e-02,  6.93359375e-02,  1.02539062e-02,  1.75781250e-01,
       -1.68945312e-01,  1.21307373e-03, -2.98828125e-01, -1.15234375e-01,
        5.66406250e-02, -

In [41]:
model.wv.similarity('woman', 'man')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


0.76640123

In [42]:
model.wv.most_similar('woman')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('man', 0.7664012908935547),
 ('girl', 0.7494640946388245),
 ('teenage_girl', 0.7336829900741577),
 ('teenager', 0.631708562374115),
 ('lady', 0.6288785934448242),
 ('teenaged_girl', 0.6141783595085144),
 ('mother', 0.607630729675293),
 ('policewoman', 0.6069462299346924),
 ('boy', 0.5975908041000366),
 ('Woman', 0.5770983695983887)]

In [43]:
model.wv.most_similar('tasty')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('delicious', 0.8730389475822449),
 ('scrumptious', 0.8007042407989502),
 ('yummy', 0.7856924533843994),
 ('flavorful', 0.7420164346694946),
 ('delectable', 0.7385421991348267),
 ('juicy_flavorful', 0.7114803791046143),
 ('appetizing', 0.7017217874526978),
 ('crunchy_salty', 0.7012300491333008),
 ('flavourful', 0.6912213563919067),
 ('flavoursome', 0.6857702732086182)]

In [44]:
model.wv.similarity('tasty', 'tast')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


0.44035053

In [45]:
#train our own w2v model using your own text corpus
import gensim
i=0
list_of_sent = []
for sent in final['Text'].values:
  filtered_sentence = []
  sent = cleanHtml(sent)
  for w in sent.split():
    for cleaned_word in cleanPunc(w).split():
      if (cleaned_word.isalpha()):
        filtered_sentence.append(cleaned_word.lower())

      else:
        continue
  list_of_sent.append(filtered_sentence)


In [46]:
print(final['Text'].values[0])

print("************************************************")
print(list_of_sent[0])

There are a number of different Switch juice flavors.  I tried Kiwi Berry, and I did like it.  There's a whole lot more than Kiwi in this can though, and while they are all natural products (meaning, not man made, I guess) I counted 5 other things besides Kiwi in there, which some of the other reviewers also pointed out.  It is carbonated - which brings out the flavor a bit more, in my opinion. It was sweet, yet they say no sugars are "added" (however, the natural products in the concoction already have 33g of sugar in them, in the 8.3 fluid ounce aluminum pop-top can).  There is 135 calories and 34g of carbs in that can too.<br /><br />So, while the flavor is good and I liked the sensation as I drank it, the marketing of the can (tall and narrow, a pretty pink, labeled as 100% of what you want, 0% of what you don't), the can makes me feel like this product is calling out to me saying, "buy me because I'm a "premium" product."  Well, I'm sorry, that's not why I buy a juice product.  I 

In [47]:

w2v_model = gensim.models.Word2Vec(list_of_sent, min_count=5, size=50, workers=3)




In [48]:
words = list(w2v_model.wv.vocab)
print(len(words))

6182


In [49]:
w2v_model.wv.most_similar('tasty')


[('satisfying', 0.8698177337646484),
 ('crunchy', 0.8653876781463623),
 ('yummy', 0.8492218852043152),
 ('chewy', 0.839055061340332),
 ('delicious', 0.8370506167411804),
 ('crisp', 0.8118793368339539),
 ('chocolatey', 0.806164026260376),
 ('healthy', 0.7992492318153381),
 ('super', 0.797649085521698),
 ('flavorful', 0.7870170474052429)]

In [50]:
  w2v_model.wv.most_similar('like')

[('prefer', 0.6939771175384521),
 ('think', 0.6807005405426025),
 ('enjoy', 0.6650040149688721),
 ('know', 0.6396976709365845),
 ('liked', 0.6289935111999512),
 ('notice', 0.6120741367340088),
 ('crave', 0.5880152583122253),
 ('actually', 0.5805429220199585),
 ('eat', 0.5786688327789307),
 ('smell', 0.5677664279937744)]

In [51]:
count_vect_feat = count_vect.get_feature_names()
count_vect_feat.index('like')
print(count_vect_feat[64055])

experience highly


In [52]:
sent_vectors = [];
for sent in list_of_sent:
  sent_vec = np.zeros(50)
  cnt_words = 0;
  for word in sent:
    try:
      vec = w2v_model.wv[word]
      sent_vec += vec
      cnt_words += 1
    except:
      pass
  sent_vec /= cnt_words
  sent_vectors.append(sent_vec)
  print(len(sent_vectors))
  print(len(sent_vectors[0]))

Streaming output truncated to the last 5000 lines.
7501
50
7502
50
7503
50
7504
50
7505
50
7506
50
7507
50
7508
50
7509
50
7510
50
7511
50
7512
50
7513
50
7514
50
7515
50
7516
50
7517
50
7518
50
7519
50
7520
50
7521
50
7522
50
7523
50
7524
50
7525
50
7526
50
7527
50
7528
50
7529
50
7530
50
7531
50
7532
50
7533
50
7534
50
7535
50
7536
50
7537
50
7538
50
7539
50
7540
50
7541
50
7542
50
7543
50
7544
50
7545
50
7546
50
7547
50
7548
50
7549
50
7550
50
7551
50
7552
50
7553
50
7554
50
7555
50
7556
50
7557
50
7558
50
7559
50
7560
50
7561
50
7562
50
7563
50
7564
50
7565
50
7566
50
7567
50
7568
50
7569
50
7570
50
7571
50
7572
50
7573
50
7574
50
7575
50
7576
50
7577
50
7578
50
7579
50
7580
50
7581
50
7582
50
7583
50
7584
50
7585
50
7586
50
7587
50
7588
50
7589
50
7590
50
7591
50
7592
50
7593
50
7594
50
7595
50
7596
50
7597
50
7598
50
7599
50
7600
50
7601
50
7602
50
7603
50
7604
50
7605
50
7606
50
7607
50
7608
50
7609
50
7610
50
7611
50
7612
50
7613
50
7614
50
7615
50
7616
50
7617
50
7618
50
7619


In [ ]:
tfidf_feat = tf_idf_vect.get_feature_names()
tfidf_sent_vectors = [];
row=0;
for sent in list_of_sent:
  sent_vec = np.zeros(50)
  weight_sum = 0;
  for work in sent:
    try:
      vec = w2v_model.wv[word]
      tfidf = final_tf_idf[row, tfidf_feat.index(word)]
      sent_vec += (vec*tf_idf)
    except:
      pass
  sent_vec /= weight_sum
  tfidf_sent_vectors.append(sent_vec)
  row += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
